In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
## load the trained model, scaler pickle, onehot
model=load_model('model.keras') ## why not working model.keras in my pc

## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
input_df=pd.DataFrame([input_data])

In [5]:
## onehot encode 'geography'
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]])
encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

e:\Projects\udemy final\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [6]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## label encode categorical values
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df=input_df.drop('Geography',axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1,40,3,60000,2,1,1,50000


In [8]:
## concatination
input_df=pd.concat([input_df,encoded_df],axis=1)

In [9]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
## scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.52749967,  0.90521949,  0.1018369 , -0.69899125, -0.26930138,
         0.80935601,  0.64395441,  0.96898085, -0.86384212,  0.99825153,
        -0.57908232, -0.57427105]])

In [11]:
## predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


array([[0.0500016]], dtype=float32)

In [12]:
prediction_probab=prediction[0][0]
prediction_probab

0.050001595

In [13]:
if prediction_probab>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
